# Sec Recon Analyses

Here we perform different analyses to test the quality of secRecon

In [1]:
import datetime
import pickle
import tqdm.notebook as tqdm

import re
import pandas as pd
import numpy as np
from Bio import Entrez
import Request_Utilis
from google_sheet import GoogleSheet
from collections import defaultdict

import matplotlib.pyplot as plt
import plotly.express as px
from venn import venn
from upsetplot import UpSet
import matplotlib.pyplot as plt
from itertools import product

# Warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
##### ----- Generate datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
Sec_Recon_SPREADSHEET_ID = '1DaAdZlvMYDqb7g31I5dw-ZCZH52Xj_W3FnQMFUzqmiQ'

# Initialize the GoogleSheet object
sec_recon_gsheet_file = GoogleSheet(Sec_Recon_SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from SecRecon
sec_genes_sheet = 'SecRecon'
ontology_sheet = 'Ontology'

sec_genes = sec_recon_gsheet_file.read_google_sheet(sec_genes_sheet)
ontology = sec_recon_gsheet_file.read_google_sheet(ontology_sheet)

### Network Analysis

In this section we'll use the networks generated in the Network_visualization notebook to visualize experimental data from a CHO High vs Low dataset

In [ ]:
# Read dataset
cho_vs_plasma_prot = pd.read_excel('Data/cho_vs_plasma/1-s2.0-S1096717624000521-mmc3.xlsx',
                                   sheet_name = 'Proteome DE proteins')

In [ ]:
# Map Mouse genes to Human genes

dict_mouse_human = dict(zip(sec_genes['MOUSE GENE SYMBOL'], sec_genes['GENE SYMBOL']))
cho_vs_plasma_prot['Human_Genes'] = cho_vs_plasma_prot['Mmus_Genes'].map(gene_dict)

In [ ]:
cho_vs_plasma_prot.Human_Genes.dropna()

In [ ]:
# Read the dictionary from the pickle file
with open('gene_dict.pkl', 'rb') as f:
    gene_dict = pickle.load(f)

### 2. Identification of secRecon genes in CRISPR CHO whole genome library

In [3]:
# Load CRISPR library dataset
guide_rna = pd.read_excel('Data/guide_rna_lib/CRISPR_a_Library_sorted_file_without_duplicates.xlsx')

# Load TFs from IPA dataset
ipa_tfs = pd.read_excel('Data/IPA_analysis/TF_secrecon_lists.xlsx', sheet_name = 'IPA_results_85-TF', usecols='B')

In [ ]:
Entrez.email = "pablodigiusto91@gmail.com"
'''
for i,row in ipa_tfs.iterrows():
    human_entrez = Request_Utilis.get_entrez_id(row['Upstream Regulator'])
    ipa_tfs.at[i, 'human_entrezid'] = human_entrez

for i,row in tqdm.tqdm(guide_rna.iterrows(), total=guide_rna.shape[0]):
    cho_entrez = Request_Utilis.get_entrez_id(row['target_name'].replace('gene-', ''), organism='Cricetulus griseus')
    guide_rna.at[i, 'cho_entrezid'] = cho_entrez
'''

In [4]:
# Find common genes between CRISPR library and secRecon
guide_rna['target_name'] = guide_rna['target_name'].str.replace('gene-', '')
common_genes_sec_recon = guide_rna[guide_rna['target_name'].isin(sec_genes['CHO GENE SYMBOL'])]

In [5]:
# Find common genes between CRISPR library and IPA Analysis
guide_rna['gene_upper'] = guide_rna['target_name'].str.replace('gene-', '').str.upper()
common_genes_ipa = guide_rna[guide_rna['gene_upper'].isin(ipa_tfs['Upstream Regulator'])]
common_genes_ipa.drop(['gene_upper'], axis=1, inplace=True)

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_2241/2727610769.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_genes_ipa.drop(['gene_upper'], axis=1, inplace=True)


In [ ]:
'''
# Remove the "NT_" prefix from the third dataset's Name column
guide_rna_nt['ID_match'] = guide_rna_nt['Name'].str.replace('NT_', '').str.replace('T_', '')

# Filter the guide_rna_nt dataset based on the Manifest_Name values
common_genes_id_match = common_genes['Manifest Name'].str.replace('NT_', '').str.replace('T_', '')

common_genes_nt = guide_rna_nt[guide_rna_nt['ID_match'].isin(common_genes_id_match)]
'''

In [8]:
# Comparison of genes in the CRISPR library and secRecon

print(f' Total genes in CRISPR library: {len(guide_rna.target_name.unique())}')
print(f' Genes in CRISPR library covered by secRecon: {len(common_genes_sec_recon.target_name.unique())}')
print(f' Total CHO genes in secRecon:',len(sec_genes['CHO GENE SYMBOL'].unique()))

 Total genes in CRISPR library: 13810
 Genes in CRISPR library covered by secRecon: 237
 Total CHO genes in secRecon: 1100


In [11]:
# Comparison of genes in the CRISPR library and IPA analysis

print(f' Total genes in CRISPR library: {len(guide_rna.target_name.unique())}')
print(f' Genes in CRISPR library covered by IPA analysis: {len(common_genes_ipa.target_name.unique())}')
print(f' Total CHO genes in IPA TFs:',len(ipa_tfs['Upstream Regulator'].unique()))

 Total genes in CRISPR library: 13810
 Genes in CRISPR library covered by IPA analysis: 26
 Total CHO genes in IPA TFs: 85


In [12]:
common_genes_sec_recon.to_excel('Data/guide_rna_lib/CRISPR_a_Library_sorted_file_without_duplicates_sec_genes.xlsx', index=False)
common_genes_ipa.to_excel('Data/guide_rna_lib/CRISPR_a_Library_sorted_file_without_duplicates_ipa_tfs.xlsx', index=False)